### Import libraries

In [7]:
# Load libraries
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
from sklearn.linear_model import LinearRegression
import socket

### Customize

In [8]:
# Customize parameters
my_input_file='0411 my_input.csv'
my_x_values=['Customer_Satisfaction', 'Training_Hours', 'Process_Efficiency']
my_y_value='Net_Profit_Margin'
my_target_profit=13
my_target_efficiency=70
my_target_training_hours=20
my_host = '127.0.0.2'

### Load data and train model

In [9]:
# load input
data=pd.read_csv(my_input_file)

# Train model to predict profitability from non-financial indicators
X = data[my_x_values]
y = data[my_y_value]
model = LinearRegression().fit(X, y)
data['Predicted_Profit_Margin'] = model.predict(X)

### Generate messages

In [10]:
# Strategic Message Generator
def generate_message(row, role):
    messages = []

    # Executive View
    if role == 'executive':
        if row['Predicted_Profit_Margin'] < my_target_profit:
            messages.append("?? Profit risk detected. Review training and customer experience strategy.")
        else:
            messages.append("Branch aligned with profit goals.")
    
    # Operations Manager View
    if role == 'operations':
        if row['Process_Efficiency'] <  my_target_efficiency:
            messages.append("?? Improve process efficiency to meet operational benchmarks.")
        else:
            messages.append("Operational efficiency is within acceptable range.")

    # HR Strategy View
    if role == 'hr':
        if row['Training_Hours'] <  my_target_training_hours:
            messages.append("?? Training investment is below threshold. Consider upskilling programs.")
        else:
            messages.append("Sufficient training hours achieved.")

    return " | ".join(messages)

### Setup dashboard app

In [11]:
# Build Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Strategy Communication Dashboard"),

    html.Label("Select Stakeholder Role:"),
    dcc.Dropdown(
        id='role_selector',
        options=[
            {'label': 'Executive', 'value': 'executive'},
            {'label': 'Operations Manager', 'value': 'operations'},
            {'label': 'HR', 'value': 'hr'}
        ],
        value='executive',
        clearable=False
    ),

    dcc.Graph(id='kpi_chart'),
    html.Div(id='strategy_messages')
])


### Callback for messaging

In [12]:
# Callback for KPI Display + Strategic Messaging 
@app.callback(
    Output('kpi_chart', 'figure'),
    Output('strategy_messages', 'children'),
    Input('role_selector', 'value')
)
def update_dashboard(role):
    # KPI Display
    kpi_map = {
        'executive': ['Revenue_Growth', 'Net_Profit_Margin', 'Predicted_Profit_Margin'],
        'operations': ['Process_Efficiency'],
        'hr': ['Training_Hours']
    }

    selected_kpis = kpi_map[role]
    df_melt = data[['Branch'] + selected_kpis].melt(id_vars='Branch', var_name='KPI', value_name='Value')
    fig = px.bar(df_melt, x='Branch', y='Value', color='KPI',
                 barmode='group', title=f"KPI View for {role.title()}")

    # Strategic Messages
    message_blocks = []
    for _, row in data.iterrows():
        message = generate_message(row, role)
        block = html.Div([
            html.Strong(row['Branch'] + ": "),
            html.Span(message)
        ])
        message_blocks.append(block)

    return fig, message_blocks


# Display Dash server address
ip_address = socket.gethostbyname(socket.gethostname())
print(f"Dash app running at http://{ip_address}:8050")

# Run the App
if __name__ == '__main__':
    
    # For external access prefix the first option  with # and remove the # from the second option
    app.run(my_host, port=8050, debug=True)
    #app.run(host='0.0.0.0', port=8050, debug=True))

Dash app running at http://192.168.1.73:8050
